In [31]:
import numpy as np
    
# class MultinomialNB:
    
#     def fit(self, X, y):
#         self.y_classes, y_counts = np.unique(y, return_counts=True)
#         self.x_classes = np.array([np.unique(x) for x in X.T])
#         self.phi_y = 1.0 * y_counts/y_counts.sum()
#         self.phi_x = self.mean_x(X, y)
#         return self
    
#     def mean_x(self, X, y):
#         return [[(X[:,j][y==k].reshape(-1,1) == self.x_classes[j]).mean(axis=0)
#                        for j in range(len(self.x_classes))]
#                       for k in self.y_classes]
    
#     def predict(self, X):
#         return np.apply_along_axis(lambda x: self.compute_probs(x), 1, X)
    
#     def compute_probs(self, x):
#         probs = np.array([self.compute_prob(x, y) for y in range(len(self.y_classes))])
#         return self.y_classes[np.argmax(probs)]
    
#     def compute_prob(self, x, y):
#         Pxy = 1
#         for j in range(len(x)):
#             i = list(self.x_classes[j]).index(x[j])
#             Pxy *= self.phi_x[y][j][i] # p(xj|y)
#         return Pxy * self.phi_y[y]
    
#     def evaluate(self, X, y):
#         return (self.predict(X) == y).mean()

class MultinomialNB:
    
    def fit(self, X, y, ls=0.01):
        self.ls = ls
        self.y_classes, y_counts = np.unique(y, return_counts=True)
        self.x_classes = [np.unique(x) for x in X.T]
        self.phi_y = 1.0 * y_counts/y_counts.sum()
        self.phi_x = self.mean_X(X, y)
        self.c_x = self.count_x(X, y)
        return self
    
    def mean_X(self, X, y):
        return [[self.ls_mean_x(X, y, k, j) for j in range(len(self.x_classes))] for k in self.y_classes]
    
    def ls_mean_x(self, X, y, k, j):
        x_data = (X[:,j][y==k].reshape(-1,1) == self.x_classes[j])
        return (x_data.sum(axis=0) + self.ls ) / (len(x_data) + (len(self.x_classes) * self.ls))
    
    def get_mean_x(self, y, j):
        return 1 + self.ls / (self.c_x[y][j] + (len(self.x_classes) * self.ls))
        
    def count_x(self, X, y):
        return [[len(X[:,j][y==k].reshape(-1,1) == self.x_classes[j])
                       for j in range(len(self.x_classes))]
                      for k in self.y_classes]

    def predict(self, X):
        return np.apply_along_axis(lambda x: self.compute_probs(x), 1, X)
    
    def compute_probs(self, x):
        probs = np.array([self.compute_prob(x, y) for y in range(len(self.y_classes))])
        return self.y_classes[np.argmax(probs)]
    
    def compute_prob(self, x, y):
        Pxy = 1
        for j in range(len(x)):
            x_clas = self.x_classes[j]
            if x[j] in x_clas:
                i = list(x_clas).index(x[j])
                p_x_j_y = self.phi_x[y][j][i] # p(xj|y)
                Pxy *= p_x_j_y
            else:
                Pxy *= get_mean_x(y, j)
        return Pxy * self.phi_y[y]
    
    def evaluate(self, X, y):
        return (self.predict(X) == y).mean()

In [32]:
from sklearn import datasets
from utils import accuracy_score
iris = datasets.load_iris()
X = iris.data  
y = iris.target
nb = MultinomialNB().fit(X, y)
nb.evaluate(X, y)

0.9666666666666667

In [33]:
nb.phi_y

array([0.33333333, 0.33333333, 0.33333333])

In [34]:
nb.phi_x

[[array([0.02018385, 0.06015188, 0.02018385, 0.08013589, 0.04016787,
         0.1001199 , 0.08013589, 0.16007194, 0.16007194, 0.06015188,
         0.02018385, 0.1001199 , 0.04016787, 0.00019984, 0.04016787,
         0.02018385, 0.00019984, 0.00019984, 0.00019984, 0.00019984,
         0.00019984, 0.00019984, 0.00019984, 0.00019984, 0.00019984,
         0.00019984, 0.00019984, 0.00019984, 0.00019984, 0.00019984,
         0.00019984, 0.00019984, 0.00019984, 0.00019984, 0.00019984]),
  array([0.00019984, 0.00019984, 0.02018385, 0.00019984, 0.00019984,
         0.00019984, 0.00019984, 0.00019984, 0.02018385, 0.12010392,
         0.08013589, 0.1001199 , 0.04016787, 0.18005596, 0.12010392,
         0.06015188, 0.06015188, 0.08013589, 0.04016787, 0.02018385,
         0.02018385, 0.02018385, 0.02018385]),
  array([2.01838529e-02, 2.01838529e-02, 4.01678657e-02, 1.40087930e-01,
         2.59992006e-01, 2.59992006e-01, 1.40087930e-01, 8.01358913e-02,
         4.01678657e-02, 1.99840128e-04, 1.998

In [28]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
clf = MultinomialNB().fit(X, Y)
print(clf.predict(X))

[1 1 1 2 2 2]


In [30]:
from sklearn import datasets
digits = datasets.load_digits()
X = digits.data
y = digits.target
MultinomialNB().fit(X, y).evaluate(X, y)

0.9755147468002225